# ABS Monthly Arrivals Departures 3401

## Python set-up

In [1]:
# analytic imports
import pandas as pd
import readabs as ra
from readabs import metacol as mc
import mgplot as mg

In [2]:
# local imports
from abs_helper import get_abs_data
import decompose
import henderson as hma

Could not import auto_arima from pmdarima


In [3]:
# pandas display settings
pd.options.display.max_rows = 999999
pd.options.display.max_columns = 999

# display charts within this notebook
SHOW = False

## Get data from ABS

In [4]:
abs_dict, meta, source, RECENT_ = get_abs_data("3401.0")
RECENT = pd.Period(RECENT_, freq="M")

plot_times = 0, RECENT
print(plot_times)

Table 3401013 has no 'Index' sheet.
Table 3401014 has no 'Index' sheet.
Table 3401015 has no 'Index' sheet.
Table 3401016 has no 'Index' sheet.
Table 3401017 has no 'Index' sheet.
(0, Period('2020-12', 'M'))


In [5]:
# data up-to-date to (year-month) ...
print(abs_dict["340101"].index[-1])

2025-04


## Headline Arrivals and Departures by Category

In [6]:
def additional_charts(title, stype, units, plt_df):
    """Additional charts for long-term and permanent residents."""

    # --- Net monthly arrivals - departures
    period = 12  # months
    net_monthly = plt_df.iloc[:, 0] - plt_df.iloc[:, 1]
    net_monthly, net_monthly_units = ra.recalibrate(net_monthly, units)
    rolling = net_monthly.rolling(period, min_periods=period).mean()
    dataset = pd.DataFrame({
        "Net Monthly Arrivals-Departures": net_monthly,
        "12m Rolling Mean": rolling,
    })
    mg.multi_start(
        dataset,
        function=mg.line_plot_finalise,
        starts=plot_times,
        title=f"{title}: Net Monthly Arrivals-Departures",
        ylabel=f"{net_monthly_units} / month",
        rfooter=source,
        lfooter=f"Australia. {stype} series. ",
        width=[1, 3],
        annotate=True,
        y0=True,
        show=SHOW,
    )

    # --- decomposition of arrivals and departures
    selector = "Seasonally Adjusted"
    selected = {}
    for column, series in plt_df.items():
        decomposed = decompose.decompose(
            series.dropna(),
            #constant_seasonal=True,
            arima_extend=True,
            ignore_years=(2020, 2021),  # COVID
        )[['Seasonally Adjusted', 'Trend']]
        selected[column] = decomposed[selector]

        mg.multi_start(
            decomposed,
            function=mg.seastrend_plot_finalise,
            starts=plot_times,
            title=f"{column} - Seasonal Decomposition",
            ylabel=f"{units} / month",
            rfooter=source,
            lfooter=f"Australia. ",
            y0=True,
            show=SHOW,
        )

    plot_trends = pd.DataFrame(selected)
    mg.multi_start(
        plot_trends,
        function=mg.line_plot_finalise,
        starts=plot_times,
        title=f"{title} movements - {selector}",
        ylabel=f"{units} / month",
        rfooter=source,
        annotate=True,
        lfooter=f"Australia. Inhouse seasonal decomposition. {selector} series. ",
        y0=True,
        show=SHOW,
    )


In [7]:
def headline() -> None:
    """Headline charts"""

    pairs = {
        # 
        "Total Overseas":(
            ("Number of movements ;  Total Arrivals ;", "340101"),
            ("Number of movements ;  Total Departures ;", "340102"),
        ),
        "Short-term residents":(
            ('Number of movements ;  Short-term Residents returning ;', '340101'),
            ('Number of movements ;  Short-term Residents departing ;', '340102'),
        ),
        "Short-term visitors":(
            ('Number of movements ;  Short-term Visitors arriving ;', '340101'),
            ('Number of movements ;  Short-term Visitors departing ;', '340102'),
        ),
        "Permanent and Long-term":(
            ('Number of movements ;  Permanent and Long-term Arrivals ;', '340101'),
            ('Number of movements ;  Permanent and Long-term Departures ;', '340102'),
        ),
    }

    stype = "Original"
    for title, plotable in pairs.items():
        plt_df = pd.DataFrame()
        for did, table in plotable:
            search = {
                did: mc.did,
                table: mc.table,
                stype: mc.stype,
            }
            _table, series_id, units = ra.find_abs_id(meta, search)
            plt_df[did.split(";")[-2].strip()] = abs_dict[table][series_id]

        plt_df, units = ra.recalibrate(plt_df, units)

        # --- Total monthly arrivals and departures
        mg.multi_start(
            plt_df,
            function=mg.line_plot_finalise,
            starts=plot_times,
            title=f"{title}: Arrivals and Departures",
            ylabel=f"{units} / month",
            rfooter=source,
            lfooter=f"Australia. {stype} series. ",
            legend=True,
            show=SHOW,
        )

        # --- Gross annual arrivals - departures
        annual = plt_df.rolling(12, min_periods=12).sum()
        net_annual = annual.iloc[:, 0] - annual.iloc[:, 1]
        net_annual, net_annual_units = ra.recalibrate(net_annual, units)
        mg.multi_start(
            net_annual,
            function=mg.line_plot_finalise,
            starts=plot_times,
            title=f"{title}: Arrivals-Departures (12m rolling sum)",
            ylabel=f"{net_annual_units} / year",
            rfooter=source,
            lfooter=f"Australia. {stype} series. ",
            annotate=True,
            y0=True,
            show=SHOW,
        )

        if title == "Permanent and Long-term":
            additional_charts(title, stype, units, plt_df)


headline()

## Individual overseas movements

In [8]:
def plot_movements() -> None:
    """Headline charts"""

    tables = "340101", "340102"
    arrivals = meta.loc[meta[mc.table] == tables[0], mc.did]
    departures = meta.loc[meta[mc.table] == tables[1], mc.did]
    movements = arrivals, departures

    stype = "Original"
    for movement, table in zip(movements, tables):
        data = abs_dict[table]
        for did in movement:
            search = {
                did: mc.did,
                table: mc.table,
                stype: mc.stype,
            }
            _table, series_id, units = ra.find_abs_id(
                meta, search, exact_match=True, verbose=False
            )
            series, units = ra.recalibrate(data[series_id], units)

            mg.multi_start(
                series,
                starts=plot_times,
                function=mg.line_plot_finalise,
                title=did.split(";")[-2],
                ylabel=f"{units} / month",
                rfooter=source,
                lfooter=f"Australia. {stype} series. ",
                annotate=True,
                y0=True,
                show=SHOW,
            )

            mg.postcovid_plot_finalise(
                series,
                title=did.split(";")[-2],
                ylabel=f"{units} / month",
                rfooter=source,
                lfooter=f"Australia. {stype} series. ",
                y0=True,
                show=SHOW,
            )


plot_movements()

## Finished

In [9]:
# watermark
%load_ext watermark
%watermark -u -t -d --iversions --watermark --machine --python --conda

Last updated: 2025-06-15 14:49:05

Python implementation: CPython
Python version       : 3.13.5
IPython version      : 9.3.0

conda environment: n/a

Compiler    : Clang 20.1.4 
OS          : Darwin
Release     : 24.5.0
Machine     : arm64
Processor   : arm
CPU cores   : 14
Architecture: 64bit

pandas : 2.3.0
readabs: 0.0.31
mgplot : 0.1.13a1

Watermark: 2.5.0



In [10]:
print("Finished")

Finished
